In [1]:
from IPython import get_ipython


  # Computational Statistics Homework 05
  Kalin Gibbons
  2020-02-13

 ## Problem 1
 Problem 2.6 on page 56 of the _Computation Statistics_ book.
 Required output to be embedded within the submitted Jupyter Notebook:
 All of the output correspoinding to a specific question must begin in a
 Markdown Cell with a heading, for example,

 **Answer to part (a):**

 --so that the grader can easily find your answer to each question.

 * (a) - (c) each. Present your estimates in the last five iterations in a format similar
   to Table 2.1. Plot the fitted model together with the observed data in a
   single plot.

 ### Problem 2.6

 Table 2.3 provides counts of a flour beetle (_Tribolium confusum_) population
 at various points in time [103]. Beetles in all stages of development were
 counted, and the food supply was carefully controlled.

 An elementary model for population growth is the logistic model given by

 $$\frac{d N}{d t} = r N \left( 1 - \frac{N}{K} \right),$$

 where $N$ is population size, $t$ is time, $r$ is a growth rate parameter,
 and $K$ is a parameter that represents the population carrying capacity of
 the environment. The solution to this differential equation is given by

 $$N_t = f(t) = \frac{K N_0}{N_0 + (K - N_0) \exp(-r t)}$$

 where $N_t$ denotes the population size at time $t$.

 * (a) Fit the logistic growth model to the flour beetle data using the
   Gauss-Newton approach to minimize the sum of squared errors between model
   predictions and observed counts.

 * (b) Fit the logistic growth model to the flour beetle data using the
   Newton-Raphson approach to minimze the sum of squared errors between model
   predictions and observed counts.

 * (c) In many population modeling applications, an assumption of lognormality
   is adopted. The simples assumption would be that the $\log N_t$ are
   independent and normally distributed with mean $\log f(t)$ and variance
   $\sigma^2$. Find the MLEs under this assumption, using both the
   Gauss-Newton and Newton-Raphson methods. Provide standard errors for your
   parameter estimates, and an estimate of the correlation between them.
   Comment.

 #### Module importing

In [2]:

import numpy as np
import pandas as pd
import sympy as sp


 #### Import the data

In [3]:
dat_dict = {'days': [0, 8, 28, 41, 63, 79, 97, 117, 135, 154],
            'population': [2, 47, 192, 256, 768, 896, 1120, 896, 1184, 1024]}
beetle_data = pd.DataFrame(dat_dict)
beetle_data


,days,population
0,0,2
1,8,47
2,28,192
3,41,256
4,63,768
5,79,896
6,97,1120
7,117,896
8,135,1184
9,154,1024


 #### Plan the solution
 The growth rate $r$ and population carrying capacity $K$ are the parameters
 which will need to be fit to the data. This is a bivariate optimization
 problem, which will require two gradients of length two, and two $2 \times 2$
 Hessian's, for the Newton-Raphson method. There are products and quotients of
 functions of differentiable variables, so we will want to use sympy to
 facilitate our calculations.

 For the Gauss Newton method, we are maximizing an objective function

 $$N_i = f(\mathbf{t_i}, \mathbf{\theta}) + \varepsilon_i$$


 for our nonlinear function $f$ and random error $\varepsilon_i$.

 We can formulate the solutions as:

 $$\Delta\mathbf{\theta} = \left( \mathbf{J^{(t)}} \cdot \mathbf{J^{(t)}} \right)^{-1} \mathbf{J^{(t)}} \mathbf{x^{(t)}}$$

 Where

 $$\Delta \mathbf{\theta} = \mathbf{\theta}^{(t + 1)} - \mathbf{\theta}^{(t)}$$

 and $\mathbf{x}^{(t)}$ is a column vector called the working response, which
 contains elements

 $$x_i^{(t)} = n_{i}^{(t)} - f(t, N_o; K, r)$$


In [4]:
# Set up the function using sympy

k, n0, r, t = sp.symbols('K N_0 r t')

num = k * n0
den = n0 + (k - n0) * sp.exp(-r * t)
n_beetles = num / den
sp.pprint(n_beetles)


K⋅N₀       
───────────────────
               -r⋅t
N₀ + (K - N₀)⋅ℯ    


In [5]:
# Create the vector $\mathbf{x}$

work_resp = beetle_data['population'] - n_beetles
work_resp


0       -K*N_0/(N_0 + (K - N_0)*exp(-r*t)) + 2
1      -K*N_0/(N_0 + (K - N_0)*exp(-r*t)) + 47
2     -K*N_0/(N_0 + (K - N_0)*exp(-r*t)) + 192
3     -K*N_0/(N_0 + (K - N_0)*exp(-r*t)) + 256
4     -K*N_0/(N_0 + (K - N_0)*exp(-r*t)) + 768
5     -K*N_0/(N_0 + (K - N_0)*exp(-r*t)) + 896
6    -K*N_0/(N_0 + (K - N_0)*exp(-r*t)) + 1120
7     -K*N_0/(N_0 + (K - N_0)*exp(-r*t)) + 896
8    -K*N_0/(N_0 + (K - N_0)*exp(-r*t)) + 1184
9    -K*N_0/(N_0 + (K - N_0)*exp(-r*t)) + 1024
Name: population, dtype: object

In [6]:
# Create our Jacobian's
f1 = sp.Function('f_1')(k, r)
f2 = sp.Function('f_2')(k, r)
fm = sp.Function('f_m')(k, r)
jacobian = sp.derive_by_array((f1, f2, fm), (k, r)).transpose()
jacobian


[[Derivative(f_1(K, r), K), Derivative(f_1(K, r), r)], [Derivative(f_2(K, r), K), Derivative(f_2(K, r), r)], [Derivative(f_m(K, r), K), Derivative(f_m(K, r), r)]]

In [7]:
sp.derive_by_array(n_beetles, (k, r))


[-K*N_0*exp(-r*t)/(N_0 + (K - N_0)*exp(-r*t))**2 + N_0/(N_0 + (K - N_0)*exp(-r*t)), K*N_0*t*(K - N_0)*exp(-r*t)/(N_0 + (K - N_0)*exp(-r*t))**2]

In [8]:
# Look's like that jacobian matches what we're looking for. Let's create the
# proper one.
all_f = (n_beetles for row in range(beetle_data.shape[0]))
all_f = []
pop_0 = beetle_data.loc[0, 'population']
for row in beetle_data.itertuples():
    iF = n_beetles.subs(t, row.days).subs(n0, pop_0)
    all_f.append(iF)
jacobian = sp.derive_by_array(all_f, (k, r)).transpose()
jacobian


[[0, 0], [-2*K*exp(-8*r)/((K - 2)*exp(-8*r) + 2)**2 + 2/((K - 2)*exp(-8*r) + 2), 16*K*(K - 2)*exp(-8*r)/((K - 2)*exp(-8*r) + 2)**2], [-2*K*exp(-28*r)/((K - 2)*exp(-28*r) + 2)**2 + 2/((K - 2)*exp(-28*r) + 2), 56*K*(K - 2)*exp(-28*r)/((K - 2)*exp(-28*r) + 2)**2], [-2*K*exp(-41*r)/((K - 2)*exp(-41*r) + 2)**2 + 2/((K - 2)*exp(-41*r) + 2), 82*K*(K - 2)*exp(-41*r)/((K - 2)*exp(-41*r) + 2)**2], [-2*K*exp(-63*r)/((K - 2)*exp(-63*r) + 2)**2 + 2/((K - 2)*exp(-63*r) + 2), 126*K*(K - 2)*exp(-63*r)/((K - 2)*exp(-63*r) + 2)**2], [-2*K*exp(-79*r)/((K - 2)*exp(-79*r) + 2)**2 + 2/((K - 2)*exp(-79*r) + 2), 158*K*(K - 2)*exp(-79*r)/((K - 2)*exp(-79*r) + 2)**2], [-2*K*exp(-97*r)/((K - 2)*exp(-97*r) + 2)**2 + 2/((K - 2)*exp(-97*r) + 2), 194*K*(K - 2)*exp(-97*r)/((K - 2)*exp(-97*r) + 2)**2], [-2*K*exp(-117*r)/((K - 2)*exp(-117*r) + 2)**2 + 2/((K - 2)*exp(-117*r) + 2), 234*K*(K - 2)*exp(-117*r)/((K - 2)*exp(-117*r) + 2)**2], [-2*K*exp(-135*r)/((K - 2)*exp(-135*r) + 2)**2 + 2/((K - 2)*exp(-135*r) + 2), 270*K*

In [9]:
# Okay, now we can create the delta function
work_resp = []
for row in beetle_data.itertuples():
    iF = row.population - n_beetles.subs([(t, row.days), (n0, pop_0)])
    work_resp.append(iF)
work_resp


[0,
 -2*K/((K - 2)*exp(-8*r) + 2) + 47,
 -2*K/((K - 2)*exp(-28*r) + 2) + 192,
 -2*K/((K - 2)*exp(-41*r) + 2) + 256,
 -2*K/((K - 2)*exp(-63*r) + 2) + 768,
 -2*K/((K - 2)*exp(-79*r) + 2) + 896,
 -2*K/((K - 2)*exp(-97*r) + 2) + 1120,
 -2*K/((K - 2)*exp(-117*r) + 2) + 896,
 -2*K/((K - 2)*exp(-135*r) + 2) + 1184,
 -2*K/((K - 2)*exp(-154*r) + 2) + 1024]

In [10]:
work_resp = sp.utilities.lambdify((k, r), work_resp, 'numpy')
work_resp


<function _lambdifygenerated(K, r)>

In [11]:
jacobian = sp.utilities.lambdify((k, r), jacobian, 'numpy')
jacobian



<function _lambdifygenerated(K, r)>

In [12]:
def gauss_newton_update(guess):
    guess = np.array(guess)
    k = guess[0]
    r = guess[1]
    J = np.array(jacobian(k, r))
    x = np.array(work_resp(k, r))
    new_guess = guess + np.linalg.inv(J.T @ J) @ J.T @ x
    return new_guess


guess = [1200, 0.17]
for i in range(12):
    guess = gauss_newton_update(guess)
    print(guess)
guess

import inspect

[9.87396868e+02 1.36665833e-01]
[1.01328182e+03 1.23433709e-01]
[1.02879375e+03 1.19347017e-01]
[1.03247009e+03 1.18322353e-01]
[1.03325327e+03 1.18054458e-01]
[1.03344705e+03 1.17983873e-01]
[1.03349736e+03 1.17965242e-01]
[1.03351058e+03 1.17960323e-01]
[1.03351407e+03 1.17959024e-01]
[1.0335150e+03 1.1795868e-01]
[1.03351524e+03 1.17958590e-01]
[1.03351530e+03 1.17958566e-01]


In [13]:
print(inspect.getsource(jacobian))

def _lambdifygenerated(K, r):
    return ([[0, 0], [-2*K*exp(-8*r)/((K - 2)*exp(-8*r) + 2)**2 + 2/((K - 2)*exp(-8*r) + 2), 16*K*(K - 2)*exp(-8*r)/((K - 2)*exp(-8*r) + 2)**2], [-2*K*exp(-28*r)/((K - 2)*exp(-28*r) + 2)**2 + 2/((K - 2)*exp(-28*r) + 2), 56*K*(K - 2)*exp(-28*r)/((K - 2)*exp(-28*r) + 2)**2], [-2*K*exp(-41*r)/((K - 2)*exp(-41*r) + 2)**2 + 2/((K - 2)*exp(-41*r) + 2), 82*K*(K - 2)*exp(-41*r)/((K - 2)*exp(-41*r) + 2)**2], [-2*K*exp(-63*r)/((K - 2)*exp(-63*r) + 2)**2 + 2/((K - 2)*exp(-63*r) + 2), 126*K*(K - 2)*exp(-63*r)/((K - 2)*exp(-63*r) + 2)**2], [-2*K*exp(-79*r)/((K - 2)*exp(-79*r) + 2)**2 + 2/((K - 2)*exp(-79*r) + 2), 158*K*(K - 2)*exp(-79*r)/((K - 2)*exp(-79*r) + 2)**2], [-2*K*exp(-97*r)/((K - 2)*exp(-97*r) + 2)**2 + 2/((K - 2)*exp(-97*r) + 2), 194*K*(K - 2)*exp(-97*r)/((K - 2)*exp(-97*r) + 2)**2], [-2*K*exp(-117*r)/((K - 2)*exp(-117*r) + 2)**2 + 2/((K - 2)*exp(-117*r) + 2), 234*K*(K - 2)*exp(-117*r)/((K - 2)*exp(-117*r) + 2)**2], [-2*K*exp(-135*r)/((K - 2)*exp(-135*r) + 2